# Quickstart

In this example, it is shown how to use ComPWA via the Python interface. We use the decay  $J/\Psi \rightarrow \gamma \pi^0 \pi^0$ to illustrate the usual workflow is:

1. Create an amplitude model for the decay.
2. Generate a Monte Carlo data sample (hit & miss) using this model. 
3. Perform a fit on the data sample using the Minuit2 interface.
4. Visualize the data and the fit result
5. Calculating fit fractions

In each step, we store the output, so that you can pick up your work at a later stage (like after performing a fit that takes several hours).

Let's go!

First, we `import` the necessary components of the expert system:

In [ ]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML)

# just a little function to print the intermediate states
def print_intermediate_states(solutions):
    from pycompwa.expertsystem.topology.graph import (
        get_intermediate_state_edges)
    print("intermediate states:")
    intermediate_states = set()
    for g in solutions:
        edge_id = get_intermediate_state_edges(g)[0]
        intermediate_states.add(g.edge_props[edge_id]['@Name'])
    print(intermediate_states)

## Step 1: Creating the amplitude model

### 1.1. Define problem set for this decay

We first define the boundary conditions of our physics problem, such as initial state, final state, formalism type, etc. and pass all of that information to the `StateTransitionManager`. This is the main user interface class of the ComPWA expert system.

In [ ]:
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

### 1.2. Preparation
Create all topology graphs using the **isobar model** (tree of two-body decays) and initialize the graphs with the initial and final state. Remember that each interaction node defines its own set of conservation laws.

The `StateTransitionManager` (STM) defines three interaction types:

| Interaction          | Strength  |
| -------------------- | --------- |
| strong               | $60$      |
| electromagnetic (EM) | $1$       |
| weak                 | $10^{-4}$ |

By default, all three are used in the preparation stage. The function `prepare_graphs()` of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph and they are grouped according to this strength.

In [ ]:
graph_interaction_settings_groups = tbd_manager.prepare_graphs()

### 1.3. Finding solutions
If you are happy with the default settings generated by the ``StateTransitionManager``, just start with solving directly!

In [ ]:
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have a lot of solutions that are actually heavily suppressed (they involve two weak decays).

In general, you can modify the dictionary returned by `prepare_graphs()` directly, but the STM also comes with functionality to globally choose the allowed interaction types (`set_allowed_interaction_types()`).

So, go ahead and **disable** the **EM** and **weak** interaction!

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Huh, what happened here? Actually, since a **$\gamma$ particle** appears in one of the interaction nodes, the expert system knows that this node **must involve EM interactions**! Because the node can be an effective interaction, the weak interaction cannot be excluded, as it contains only a subset of conservation laws.

Since only the strong interaction was supposed to be used, this results in a warning and the STM automatically corrects the mistake.

Once the EM interaction is included, this warning disappears. Be aware, however, that the EM interaction is now available globally. Hence, there now might be solutions in which both nodes are electromagnetic.

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Great! Now we selected only the strongest contributions. Be aware, though, that there are more effects that can suppress certain decays. For example branching ratios. In this example $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay                               | branching ratio |
| ----------------------------------- | --------------- |
| $$\omega \rightarrow \gamma+\pi^0$$ | 0.0828          |
| $$\rho^0 \rightarrow \gamma+\pi^0$$ | 0.0006          |

Unfortunately, the $\rho^0$ mainly decays into $\pi+\pi$, not $\gamma+\pi^0$ and is therefore suppressed. This information is currently not known to the expert system, but it is possible to hand the expert system a list of allowed intermediate states:

In [ ]:
# particles are found by name comparison, i.e. f2 will find all f2's and f all f's independent of their spin
tbd_manager.allowed_intermediate_particles = ['f']

(solutions, violated_rules) = tbd_manager.find_solutions(
        graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have selected all amplitudes that involve **f** states. The appearing warnings notify the user that the solution space is only partial. For certain lines in the graph, no suitable particle was found (since only f states were allowed).

At this point, we are all set to generate some data using this amplitude model!

In [ ]:
xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

Have a look through the sections of the resulting ``model.xml`` file to see what you recognize from the problem set defined so far.

## Step 2: Generate a data sample

In this section, we will use the amplitude model created above to generate a data sample via hit & miss Monte Carlo.

Using this amplitude model in the C++ side of ComPWA is simple. The `create_helicity_kinematics()` helper function builds a ``Kinematics`` object following the specifications in the XML model file. The second argument of this function is a particle list, which can be created with the `read_particles()` function. The created ``Kinematics`` object can be used to generate a phase space sample as well.

An ``Intensity`` object can be created with the `create_intensity()` function. Because the Intensities constructed by the Builder are automatically normalized, you have to pass this function a phase space sample as an argument. The ``Kinematics`` instance is then updated with the needed subsystems of the decay topology during the creation of the ``Intensity`` object.

Now all building blocks for generating our data sample are at hand.

In [ ]:
import pycompwa.ui as pwa

particle_list = pwa.read_particles('model.xml')

kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)

generator = pwa.EvtGenGenerator(kinematics.get_particle_state_transition_kinematics_info())

random_generator = pwa.StdUniformRealGenerator(12345)

phsp_sample = pwa.generate_phsp(100000, generator, random_generator)

intensity_builder = pwa.IntensityBuilderXML('model.xml', particle_list, kinematics, phsp_sample)

intensity = intensity_builder.create_intensity()

data_sample = pwa.generate(10000, kinematics, generator, intensity, random_generator)

### 2.1 Writing and reading data samples

You may want to write the generated data and phase space samples to disk, so that you don't have to perform this step each time you revisit the fit. ComPWA provides functions for ROOT and ASCII files, that can be used as follows:

In [ ]:
# to ROOT
pwa.write_root_data(event_list=data_sample, output_file='generated_data.root')
pwa.write_root_data(event_list=phsp_sample, output_file='generated_phsp.root')
# to ASCII
pwa.write_ascii_data(event_list=data_sample, output_file='generated_data.dat')
pwa.write_ascii_data(event_list=phsp_sample, output_file='generated_phsp.dat')

Importing syntax parallels that:

In [ ]:
imported_data = pwa.read_root_data(input_file='generated_data.root')
print('Imported events:', len(imported_data.events))
first_event = imported_data.events[0]
print('First event:')
print(first_event.four_momenta)
print(first_event.weight)

Note that the ASCII (`.dat`) files contain a header section that informs you about the final state of the file. You will have to prepend this section to the file yourself if you want to use a data sample from another framework.

In [ ]:
with open('generated_data.dat') as f:
    for _ in range(5):
        print(f.readline()[:-1])

## Step 3: Fitting

An **Intensity** object behaves just like a mathematical function that takes a `DataSet` as an argument and returns a list of intensities (real numbers).

To perform a fit, first create an estimator of your choice. An estimator generally needs:

- an `Intensity` instance
- a `DataSet` (to which the intensity is fitted)
- optionally: a phase space `DataSet` (which is used to normalize the `Intensity`)

A phase space sample can be generated via the `generate_phsp()` function (see above). Since our `Intensity` is already normalized, this is not needed here.
The data samples can be converted to `DataSet` using the `convert()` method of `Kinematics`.

In [ ]:
data_set = kinematics.convert(data_sample)

Now we can create an **estimator**. Remember that an estimator indicates how well a set of model parameters describes a given data set best. In this example, we use an unbinned log likelihood estimator. The fit calculations are then performed the calculations with the `FunctionTree` back-end.

In [ ]:
estimator, initial_parameters = pwa.create_unbinned_log_likelihood_function_tree_estimator(intensity, 
                                                                                           data_set)

Notice that you not only receive a estimator object, but also a list of fit parameters (`FitParameterList`). You use this list of fit parameters to initialize the optimizer later on. They contain the following info:

- the initial values of the parameters
- fix parameters
- define boundaries
- define errors, which can give certain optimizers hints on the step size

These fit parameters are initialized with the values stated in the XML file or with default values if unspecified. But can be changed easily in Python as well, like fixing certain parameters.

In [ ]:
from pprint import pprint
pprint(initial_parameters)
print("\nthis parameter is initially not fixed:")
print(initial_parameters[8])
initial_parameters[7].is_fixed = True
initial_parameters[8].is_fixed = True
print("\nand now it is fixed:")
print(initial_parameters[8])

To make the fit a bit more interesting, we modify one of the parameters to a different initial value than the true value.

In [ ]:
print("before:")
print(initial_parameters[12])
initial_parameters[12].value = 2.0
print("after:")
print(initial_parameters[12])

Now it's time to start up a set up a fit. This is quite simple: just create an optimizer instance of your choice, here Minuit2 (`MinuitIF`), and call its `optimize()` method to start the fitting process.

In [ ]:
optimizer = pwa.MinuitIF()
fit_result = optimizer.optimize(estimator, initial_parameters)

In [ ]:
fit_result.fit_duration_in_seconds

Let's check if the fit parameters are "close to" the true values.

In [ ]:
print("this should be close to 1.0 again:")
print(fit_result.final_parameters[12].value, 
      "+-", 
      fit_result.final_parameters[12].error)

**Important**: Note that the intensity instance still needs to be notified about this optimal set of parameters. They can be applied with the `updateParametersFrom()` function.

In [ ]:
intensity.updateParametersFrom(fit_result.final_parameters)

Again, you can store this fit result to disk and pick it up again:

In [ ]:
fit_result.write('fit_result.xml')
imported_fit_result = pwa.load('fit_result.xml')
imported_fit_result.final_parameters[12]

## Step 4: Visualization

Let's go ahead and make a Dalitz plot of the data sample and our fit result. It is easiest to visualize your data with [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). This section illustrates how to do this in combination with ComPWA's functionality.

Since our model only includes one specific decay topology, the `HelicityKinematics` class only generates the kinematic variables needed to evaluate the `Intensity`. In order to make a Dalitz plot, the kinematic variables from the other subsystems are also needed. They can be created with `create_all_subsystems()`. The conversion from the event samples have to be performed again, though:

In [ ]:
kinematics.create_all_subsystems()
data_set = kinematics.convert(data_sample)
phsp_set = kinematics.convert(phsp_sample)

The `data_set` and `phsp_set` instances are `DataSet` objects can cannot be understood by pandas. Their data members, however, easily allow us to make the conversion:

In [ ]:
import pandas as pd
frame_data = pd.DataFrame(data_set.data)
frame_phsp = pd.DataFrame(phsp_set.data)
frame_data['weights'] = data_set.weights
frame_data

We added a weights column here as well, though in this case, all data weights are unity.

To change the number IDs here to the original particle names, you can extract a mapping from the `Kinematics` object:

In [ ]:
transition_info = kinematics.get_particle_state_transition_kinematics_info()
id_to_name = transition_info.get_final_state_id_to_name_mapping()
name_to_id = {val: key for key, val in id_to_name.items()}
def replace_ids(title):
    """Just some replace lambda"""
    for id, name in id_to_name.items():
       title = title.replace(str(id), name)
    return title

How to visualize the fit? This is where the phase space comes in again. By attributing an intensity as a weight to each phase space point, we get a point distribution of the fit model over the space spanned by the kinematic variables.

In [ ]:
intensity_set = intensity.evaluate(phsp_set.data)
frame_phsp['weights'] = intensity_set

Now you can apply the usual `matplotlib.pyplot` tools to plot the distributions.

In [ ]:
import matplotlib.pyplot as plt
def plot_1d_comparison(name, **kwargs):
    """Helper function for comparing the 1D distributions of fit and data"""
    frame_data[name].hist(bins=100, density=True, alpha=.5, label='data', **kwargs);
    frame_phsp[name].hist(bins=100, weights=frame_phsp['weights'],
                          density=True, histtype='step', color='red', label='fit', **kwargs);
    plt.ylabel('normalized intensity')
    title = replace_ids(name)
    plt.xlabel(title)

In [ ]:
plot_1d_comparison('theta_2_4_vs_3')
plt.legend();

In [ ]:
plot_1d_comparison('mSq_(3,4)')
plt.legend();

Similarly, we can easily create a Dalitz plot:

In [ ]:
import matplotlib.pyplot as plt
def dalitz_plot(frame, mass_x, mass_y, **kwargs):
    """Helper function to create a Dalitz plot with meaningful axis titles"""
    plt.hist2d(
        frame[mass_x],
        frame[mass_y],
        weights=frame['weights'],
        **kwargs
    );
    plt.xlabel(replace_ids(mass_x))
    plt.ylabel(replace_ids(mass_y))

In [ ]:
from itertools import combinations
variable_names = data_set.data.keys()
invariant_masses = [var for var in variable_names
                    if var.startswith('mSq_')
                    and not var.endswith('(2,3,4)')]
for comb in combinations(invariant_masses, 2):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    plt.sca(axs[0])
    dalitz_plot(frame_data, *comb, bins=100)
    axs[0].set_title('data')
    plt.sca(axs[1])
    dalitz_plot(frame_phsp, *comb, bins=100)
    axs[1].set_title('fit')
    plt.show()

## Step 5: Fit Fractions

Fit fractions can be calculated using the ``fit_fractions_with_propagated_errors()`` function. It requires amplitude or ``Intensity`` components that can be extracted from the ``IntensityBuilderXML`` instance. A nested list of the component names has to be specified as well. These are the names specified in the component XML attributes. If the inner lists contain more than one component, these components will be added coherently. In this way you can calculate your own customized fit fractions.

All registered component names can be retrieved via ``get_all_component_names()``:

In [ ]:
intensity_builder.get_all_component_names()

Now specify which components you want to get from the builder.

In [ ]:
components = intensity_builder.create_intensity_components([
    ['coherent_J/psi_-1_to_gamma_-1+pi0_0+pi0_0'],
    ['J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;'],
    ['J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;'],    
    ['J/psi_-1_to_f2(1270)_-1+gamma_-1;f2(1270)_-1_to_pi0_0+pi0_0;',
     'J/psi_-1_to_f2(1270)_-2+gamma_-1;f2(1270)_-2_to_pi0_0+pi0_0;',
     'J/psi_-1_to_f2(1270)_0+gamma_-1;f2(1270)_0_to_pi0_0+pi0_0;'],
])

The last step is to call the actual fit fraction calculation function ``fit_fractions_with_propagated_errors()``. Here, the first required argument is a list of component pairs. Each pair resembles the nominator and denominator of a fit fraction.

In [ ]:
fit_fractions = pwa.fit_fractions_with_propagated_errors(
    [(components[1], components[0]),
     (components[2], components[0]),
     (components[3], components[0]),
     (components[4], components[0]),
    ], phsp_set, fit_result)

In [ ]:
print(fit_fractions)

That's it. You can check some of the other examples to learn about more detailed features of ComPWA.

And give us feedback or [contribute](https://compwa.github.io/contribute.html)! ;)